# Worksheet 08

Name:  Hasan Mustafabayli
UID: U15717216

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [76]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.90207059813477, 4.947717144576793, 5.676617165736097, 4.73921628587266, 4.9448632878988406, 3.6088614347535586, 4.532227763472874, 5.49399345679775, 3.764790853170411, 4.217748480059462]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [77]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[9.145402875311772, 8.229967780139495, 7.94628646842143, 9.856855624511967, 8.431171807411804, 6.899024495265549, 8.351280806520323, 7.989758228728474, 6.8534428258950495, 8.045451673092222]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [78]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[4.217748480059462, 3.764790853170411, 8.045451673092222, 6.8534428258950495, 7.989758228728474, 8.351280806520323, 5.49399345679775, 4.532227763472874, 6.899024495265549, 3.6088614347535586]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]

print(s2)

prob_s = [len(s1) / (len(s1) + len(s2)), len(s2) / (len(s1) + len(s2))]
mean = [sum(s1)/len(s1), sum(s2)/len(s2)]
var = [sum(map(lambda x: (x - mean[0])**2, s1)) / len(s1), sum(map(lambda x: (x - mean[1])**2, s2)) / len(s2)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [79]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s2) + len(s1)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , 
       sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2)  ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[8.045451673092222, 6.8534428258950495, 7.989758228728474, 8.351280806520323, 6.899024495265549]
[4.217748480059462, 3.764790853170411, 5.49399345679775, 4.532227763472874, 3.6088614347535586]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 7.627791605900323,  mean_2 = 4.323524397650812
var_1 = 0.39192278516414747,  var_2 = 0.4495339564665565


/Users/hasanmustafabayli/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [85]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
       
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
       
        prob_s[j]

   
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    
    prob_s0_x.append( (pdf_i[0]*prob_s[0])/prob_x )
    prob_s1_x.append( (pdf_i[1]*prob_s[1])/prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  4.217748480059462
probability of observing that point if it came from cluster 0 =  3.712805917762503e-17
probability of observing that point if it came from cluster 1 =  0.02568100143196745
point =  3.764790853170411
probability of observing that point if it came from cluster 0 =  8.179570535454288e-22
probability of observing that point if it came from cluster 1 =  0.02586047697392014
point =  8.045451673092222
probability of observing that point if it came from cluster 0 =  0.5769013621177081
probability of observing that point if it came from cluster 1 =  0.023355213304780246
point =  6.8534428258950495
probability of observing that point if it came from cluster 0 =  0.14457211454585064
probability of observing that point if it came from cluster 1 =  0.024222971173943137
point =  7.989758228728474
probability of observing that point if it came from cluster 0 =  0.6644794743988108
probability of observing that point if it came from cluster 1 =  0.023398316252070988
point =  

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [81]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x),  sum(prob_s1_x)/ len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),  sum(prob_s1_x)/ len(prob_s1_x) ]
var = [sum([(x - mean[0]) ** 2 * prob for x, prob in zip(data, prob_s0_x)]) / sum(prob_s0_x),
       sum([(x - mean[1]) ** 2 * prob for x, prob in zip(data, prob_s1_x)]) / sum(prob_s1_x)]


print("P(S_1) = " + str(prob_c[0]) + ",  P(S_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5000011266826824,  P(S_2) = 0.49999887331731774
mean_1 = 7.6277864736176975,  mean_2 = 0.49999887331731774
var_1 = 0.3919330995021539,  var_2 = 15.068862830480922


g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [84]:
import numpy as np
from scipy.stats import norm


num_clusters = 2
data_points = [1.2, 2.3, 0.8, 3.1, 1.8, 2.5]  

# Initialize parameters
cluster_means = [1.0, 2.5]  
cluster_variances = [0.2, 0.3]  
cluster_prior_probabilities = [0.4, 0.6]  

updated_prob_cluster_0 = []  
updated_prob_cluster_1 = []  

for data_point in data_points:
    likelihoods = []

    for cluster_index in range(num_clusters):
       
        likelihoods.append(norm.pdf(data_point, cluster_means[cluster_index], np.sqrt(cluster_variances[cluster_index])))

  
    marginal_likelihood = cluster_prior_probabilities[0] * likelihoods[0] + cluster_prior_probabilities[1] * likelihoods[1]

  
    updated_prob_cluster_0.append(likelihoods[0] * cluster_prior_probabilities[0] / marginal_likelihood)
    updated_prob_cluster_1.append(likelihoods[1] * cluster_prior_probabilities[1] / marginal_likelihood)


for old_prob_0, old_prob_1, new_prob_0, new_prob_1 in zip(prob_s0_x, prob_s1_x, updated_prob_cluster_0, updated_prob_cluster_1):
    print(f"Original P(S_0 | X_i): {old_prob_0}, Updated P(S_0 | X_i): {new_prob_0}")
    print(f"Original P(S_1 | X_i): {old_prob_1}, Updated P(S_1 | X_i): {new_prob_1}")
    print()




Original P(S_0 | X_i): 4.292142559972375e-17, Updated P(S_0 | X_i): 0.9251128982364694
Original P(S_1 | X_i): 1.0, Updated P(S_1 | X_i): 0.07488710176353061

Original P(S_0 | X_i): 1.9901530418369836e-21, Updated P(S_0 | X_i): 0.01260388878438572
Original P(S_1 | X_i): 1.0, Updated P(S_1 | X_i): 0.9873961112156142

Original P(S_0 | X_i): 0.9999999999999981, Updated P(S_0 | X_i): 0.9891634414929742
Original P(S_1 | X_i): 2.0020962816721546e-15, Updated P(S_1 | X_i): 0.010836558507025732

Original P(S_0 | X_i): 0.9999991863810624, Updated P(S_0 | X_i): 2.4233931269467076e-05
Original P(S_1 | X_i): 8.136189375860026e-07, Updated P(S_1 | X_i): 0.9999757660687305

Original P(S_0 | X_i): 0.9999999999999951, Updated P(S_0 | X_i): 0.27168983851317074
Original P(S_1 | X_i): 4.811191555690879e-15, Updated P(S_1 | X_i): 0.7283101614868293

Original P(S_0 | X_i): 1.0, Updated P(S_0 | X_i): 0.002936100398175322
Original P(S_1 | X_i): 1.770406148341859e-17, Updated P(S_1 | X_i): 0.9970638996018247



h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [83]:

cluster_assignments = []

for prob_0, prob_1 in zip(updated_prob_cluster_0, updated_prob_cluster_1):
    if prob_0 > prob_1:
        cluster_assignments.append(0)
    else:
        cluster_assignments.append(1)


for data_point, cluster_label in zip(data_points, cluster_assignments):
    print(f"Data point {data_point} is assigned to cluster {cluster_label}")



Data point 1.2 is assigned to cluster 0
Data point 2.3 is assigned to cluster 1
Data point 0.8 is assigned to cluster 0
Data point 3.1 is assigned to cluster 1
Data point 1.8 is assigned to cluster 1
Data point 2.5 is assigned to cluster 1
